In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))

In [7]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [8]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],NaN
1,JANUARY,7.0,The Legend of La Llorona,Saban Films,Patricia Harris Seeley (director/screenplay); ...,[3],NaN
2,JANUARY,7.0,The Commando,Saban Films,Asif Akbar (director); Koji Steven Sakai (scre...,[4],NaN
3,JANUARY,7.0,American Siege,Vertical Entertainment,Edward Drake (director/screenplay); Timothy V....,[5],NaN
4,JANUARY,14.0,Scream,Paramount Pictures / Spyglass Media Group,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[6],NaN
...,...,...,...,...,...,...,...
327,DECEMBER,30.0,"Alice, Darling",Lionsgate / Elevation Pictures,Mary Nighy (director); Alanna Francis (screenp...,NaN,[263]
328,DECEMBER,NaN,NaN,NaN,NaN,NaN,NaN
329,DECEMBER,NaN,NaN,NaN,NaN,NaN,NaN
330,DECEMBER,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_2022 = df[['Title']]

In [10]:
df_2022

,Title
0,The 355
1,The Legend of La Llorona
2,The Commando
3,American Siege
4,Scream
...,...
327,"Alice, Darling"
328,NaN
329,NaN
330,NaN


In [11]:
!pip install tmdbv3api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from tmdbv3api import TMDb
import json
tmdb = TMDb()
tmdb.api_key = '2d71ac9f8f76e2fe996cdc92cab9f9cd'

In [13]:
from tmdbv3api import Movie
import json
tmdb_movie = Movie() 

In [14]:
def ids(x):
  result = tmdb_movie.search(x)
  if not result:
    return np.NaN
  else:
    return int(result[0].id)

In [15]:
df_2022['id'] = df_2022['Title'].map(lambda x: ids(str(x)))

<ipython-input-15-7a6ad77ada03>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['id'] = df_2022['Title'].map(lambda x: ids(str(x)))


In [16]:
df_2022

,Title,id
0,The 355,522016
1,The Legend of La Llorona,739890
2,The Commando,753232
3,American Siege,766907
4,Scream,646385
...,...,...
327,"Alice, Darling",850028
328,NaN,647808
329,NaN,647808
330,NaN,647808


In [17]:
df_2022[df_2022['id'].isnull()]

,Title,id


In [19]:
df_2022['id'] = df_2022['id'].astype('int')

<ipython-input-19-c335090176be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['id'] = df_2022['id'].astype('int')


In [20]:
def description(movie_id):
        description =""
    
        movie_details = tmdb_movie.details(movie_id)
        if movie_details['overview'] and movie_details['tagline']:
          return movie_details['overview'] + "@#$" + movie_details['tagline']
        elif movie_details['overview'] and not movie_details['tagline']:
          return movie_details['overview'] + "@#$" + ""
        elif not movie_details['overview'] and movie_details['tagline']:
          return ""+ "@#$" + movie_details['tagline']
        else:
          return "@#$"

In [21]:
df_2022['description'] = df_2022['id'].map(lambda x: description(int(x)))

<ipython-input-21-ffe63254e172>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['description'] = df_2022['id'].map(lambda x: description(int(x)))


In [22]:
df_2022

,Title,id,description
0,The 355,522016,"A group of top female agents from American, Br..."
1,The Legend of La Llorona,739890,"While sleeping, a group of campers hear an omi..."
2,The Commando,753232,An elite DEA agent returns home after a failed...
3,American Siege,766907,An ex-NYPD officer-turned-sheriff of a small r...
4,Scream,646385,Twenty-five years after a streak of brutal mur...
...,...,...,...
327,"Alice, Darling",850028,A young woman trapped in an abusive relationsh...
328,NaN,647808,Catherine Tate's iconic character Nan hits the...
329,NaN,647808,Catherine Tate's iconic character Nan hits the...
330,NaN,647808,Catherine Tate's iconic character Nan hits the...


In [23]:
df_2022['description'] = df_2022['description'].apply(lambda x: x.split("@#$"))

<ipython-input-23-d3bf0b26d852>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['description'] = df_2022['description'].apply(lambda x: x.split("@#$"))


In [24]:
df_2022

,Title,id,description
0,The 355,522016,"[A group of top female agents from American, B..."
1,The Legend of La Llorona,739890,"[While sleeping, a group of campers hear an om..."
2,The Commando,753232,[An elite DEA agent returns home after a faile...
3,American Siege,766907,[An ex-NYPD officer-turned-sheriff of a small ...
4,Scream,646385,[Twenty-five years after a streak of brutal mu...
...,...,...,...
327,"Alice, Darling",850028,[A young woman trapped in an abusive relations...
328,NaN,647808,[Catherine Tate's iconic character Nan hits th...
329,NaN,647808,[Catherine Tate's iconic character Nan hits th...
330,NaN,647808,[Catherine Tate's iconic character Nan hits th...


In [25]:
df_2022['overview'] = df_2022['description'].map(lambda x: x[0])
df_2022['tagline'] = df_2022['description'].map(lambda x: x[1])

<ipython-input-25-8428739a7fe4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['overview'] = df_2022['description'].map(lambda x: x[0])
<ipython-input-25-8428739a7fe4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['tagline'] = df_2022['description'].map(lambda x: x[1])


In [26]:
df_2022

,Title,id,description,overview,tagline
0,The 355,522016,"[A group of top female agents from American, B...","A group of top female agents from American, Br...",Work together or die alone.
1,The Legend of La Llorona,739890,"[While sleeping, a group of campers hear an om...","While sleeping, a group of campers hear an omi...",
2,The Commando,753232,[An elite DEA agent returns home after a faile...,An elite DEA agent returns home after a failed...,Only one will survive.
3,American Siege,766907,[An ex-NYPD officer-turned-sheriff of a small ...,An ex-NYPD officer-turned-sheriff of a small r...,No one walks away.
4,Scream,646385,[Twenty-five years after a streak of brutal mu...,Twenty-five years after a streak of brutal mur...,It's always someone you know.
...,...,...,...,...,...
327,"Alice, Darling",850028,[A young woman trapped in an abusive relations...,A young woman trapped in an abusive relationsh...,Sometimes the hardest thing to see is the truth.
328,NaN,647808,[Catherine Tate's iconic character Nan hits th...,Catherine Tate's iconic character Nan hits the...,A coming of age movie.
329,NaN,647808,[Catherine Tate's iconic character Nan hits th...,Catherine Tate's iconic character Nan hits the...,A coming of age movie.
330,NaN,647808,[Catherine Tate's iconic character Nan hits th...,Catherine Tate's iconic character Nan hits the...,A coming of age movie.


In [27]:
df_2022['comb'] = df_2022['overview'] + df_2022['tagline']
df_2022['comb'] = df_2022['comb'].fillna('')

<ipython-input-27-a988c9ae4fc7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['comb'] = df_2022['overview'] + df_2022['tagline']
<ipython-input-27-a988c9ae4fc7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['comb'] = df_2022['comb'].fillna('')


In [28]:
df_2022 = df_2022.rename(columns={'Title':'title'})

In [29]:
df_2022 = df_2022.loc[:,['id','overview','tagline','title','comb']]

In [30]:
df_2022

,id,overview,tagline,title,comb
0,522016,"A group of top female agents from American, Br...",Work together or die alone.,The 355,"A group of top female agents from American, Br..."
1,739890,"While sleeping, a group of campers hear an omi...",,The Legend of La Llorona,"While sleeping, a group of campers hear an omi..."
2,753232,An elite DEA agent returns home after a failed...,Only one will survive.,The Commando,An elite DEA agent returns home after a failed...
3,766907,An ex-NYPD officer-turned-sheriff of a small r...,No one walks away.,American Siege,An ex-NYPD officer-turned-sheriff of a small r...
4,646385,Twenty-five years after a streak of brutal mur...,It's always someone you know.,Scream,Twenty-five years after a streak of brutal mur...
...,...,...,...,...,...
327,850028,A young woman trapped in an abusive relationsh...,Sometimes the hardest thing to see is the truth.,"Alice, Darling",A young woman trapped in an abusive relationsh...
328,647808,Catherine Tate's iconic character Nan hits the...,A coming of age movie.,NaN,Catherine Tate's iconic character Nan hits the...
329,647808,Catherine Tate's iconic character Nan hits the...,A coming of age movie.,NaN,Catherine Tate's iconic character Nan hits the...
330,647808,Catherine Tate's iconic character Nan hits the...,A coming of age movie.,NaN,Catherine Tate's iconic character Nan hits the...


In [31]:
df_2022.to_csv('desc2022.csv',index=False)

In [32]:
from google.colab import files
files.download("desc2022.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>